## Day 9: Rope Bridge

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day09/puzzle.ipynb)

Actually, the input can be a valid F# code.
```fsharp
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
All we have to do is save the input as fsx file, define corresponding function and then load the input so that it will execute.

The functions are going to mutate some global state. For better control, we can define the actual logic functions in an immutable manner which can be called by functions `R`, `U`, `L` and `D`.

The `Point` type is defined in the `common.fsx` script. Effectively it is a wrapper for `int*int`, but also provides `(+)` and `(-)` operators. Generally speaking, it is both a Point and a Vector, depending on the context.

In [92]:
#load "../common/common.fsx"

type State = { Head : Point; Tail : Point }
with static member Default = { Head = Point(0,0); Tail = Point(0,0) }

This is going to be handy to simulate all the movements as some state changes at each steps. But it is important to make sure that each motion produces a sequence of states. 

In [93]:
let move' (Point(x,y)) (state : State) =
    if (abs x > 1 || abs y > 1) then invalidArg "x,y" ""
    let newHead = state.Head + Point(x,y)
    let (Point(dx, dy)) = newHead - state.Tail
    if (abs dx > 1 || abs dy > 1)
    then { Head = newHead; Tail = state.Tail + Point(sign dx, sign dy) }
    else { state with Head = newHead }
let moveMany' v count state = 
    (List.replicate count v, state)
    ||> List.mapFoldBack (fun step prevState -> let s = move' step prevState in s,s) 
    |> fst
let R' steps state = 
    moveMany' (Point(0,1)) steps state
let L' steps state = 
    moveMany' (Point(0,-1)) steps state
let U' steps state = 
    moveMany' (Point(1,0)) steps state
let D' steps state = 
    moveMany' (Point(-1,0)) steps state

Define the global mutable list of states and mutating functions

In [94]:
let mutable states = []
let R steps = states <- (R' steps (List.head states)) @ states
let L steps = states <- (L' steps (List.head states)) @ states
let U steps = states <- (U' steps (List.head states)) @ states
let D steps = states <- (D' steps (List.head states)) @ states

In [95]:
#load "../common/matrixFormatting.fsx"
let pointsToDisplayable (points : Point array) = 
    let maxi = points |> Array.map (fun (Point(x,_)) -> x) |> Array.max
    let maxj = points |> Array.map (fun (Point(_,y)) -> y) |> Array.max
    let mini = points |> Array.map (fun (Point(x,_)) -> x) |> Array.min
    let minj = points |> Array.map (fun (Point(_,y)) -> y) |> Array.min
    let matrix = Array2D.createBased mini minj (maxi - mini + 1) (maxj - minj + 1) System.Drawing.Color.Transparent
    points |> Array.iter (fun (Point(x,y)) -> matrix[x(*mirror*),y] <- System.Drawing.Color.Red)
    matrix |> Array2D.rebase |> toDisplayable id

With these functions defined, we can now execute the sample input and process the states history.

In [96]:
states <- [ State.Default ]
#load "data_sample1.fsx"
states |> List.distinctBy (fun s -> s.Tail) |> List.length |> display

states |> List.toArray |> Array.map (fun s -> s.Tail)
|> Array.distinct |> pointsToDisplayable

13

For the actual input we are about to reset the state first:

In [97]:
states <- [ State.Default ]
#load "data_actual.fsx"
states |> List.distinctBy (fun s -> s.Tail) |> List.length |> display

states |> List.toArray |> Array.map (fun s -> s.Tail)
|> Array.distinct |> pointsToDisplayable |> withSettings { Width = 700 }

6384